In [1]:
from datetime import datetime
import numpy as np
import numpy as numpy
import pandas as pd
import pylab
import calendar
from scipy import stats
import seaborn as sns
from sklearn import model_selection, preprocessing
from scipy.stats import kendalltau
from sklearn.metrics import mean_squared_error
import warnings
import matplotlib.pyplot as plt
import pandas
## Keras comes here
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout, BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from keras import callbacks
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [2]:
print('Loading train, prop and sample data')
train = pd.read_csv("/Users/vicky/Documents/kaggle_data/train_2016_v2.csv", parse_dates=["transactiondate"])
prop = pd.read_csv('/Users/vicky/Documents/kaggle_data/properties 2016.csv')
sample = pd.read_csv('/Users/vicky/Documents/kaggle_data/sample_submission.csv')

Loading train, prop and sample data


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:

print('Fitting Label Encoder on properties')
for c in prop.columns:
    prop[c]=prop[c].fillna(-1)
    if prop[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(prop[c].values))
        prop[c] = lbl.transform(list(prop[c].values))

Fitting Label Encoder on properties


In [4]:
print('Creating training set:')
df_train = train.merge(prop, how='left', on='parcelid')


Creating training set:


In [8]:
df_train["transactiondate"] = pd.to_datetime(df_train["transactiondate"])
df_train['transactiondate_quarter'] = df_train['transactiondate'].dt.quarter

basedate = pd.to_datetime('2015-11-15').toordinal()
df_train['cos_season'] = \
        ( (pd.to_datetime(df_train['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
          (2*np.pi/365.25) ).apply(np.cos)
df_train['sin_season'] = \
        ( (pd.to_datetime(df_train['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
          (2*np.pi/365.25) ).apply(np.sin)
df_train = df_train.drop(['parcelid', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode',
                             'fireplacecnt', 'fireplaceflag'], axis=1)

In [9]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [10]:
Values = df_train.values
#test_values = df_test.values
values = Values.astype('float32')
#test_values = test_values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
#scaled_test = scaler.fit_transform(test_values)
reframed = series_to_supervised(scaled, 1, 1)
#reframed_test = series_to_supervised(scaled,1,1)
reframed.drop(reframed.columns[58:116], axis=1, inplace=True)

train_X, train_Y = reframed.iloc[:80000,:-1], reframed.iloc[:80000,-1]
test_X, test_Y = reframed.iloc[80000:,:-1], reframed.iloc[80000:,-1]

train_X = np.array(train_X)
train_Y = np.array(train_Y)
test_X = np.array(test_X)
test_Y = np.array(test_Y)

In [11]:
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape)

(80000, 1, 57) (80000,) (10274, 1, 57) (10274,)


In [12]:
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
model.fit(train_X, train_Y, epochs=50, batch_size=72, validation_data=(test_X, test_Y), verbose=2, shuffle=False)
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
inv_yhat = np.concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
test_y = test_Y.reshape((len(test_Y), 1))
inv_y = np.concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Train on 80000 samples, validate on 10274 samples
Epoch 1/50
 - 6s - loss: 0.0128 - val_loss: 0.0439
Epoch 2/50
 - 5s - loss: 0.0088 - val_loss: 0.0414
Epoch 3/50
 - 5s - loss: 0.0083 - val_loss: 0.0380
Epoch 4/50
 - 6s - loss: 0.0081 - val_loss: 0.0333
Epoch 5/50
 - 6s - loss: 0.0080 - val_loss: 0.0282
Epoch 6/50
 - 6s - loss: 0.0079 - val_loss: 0.0238
Epoch 7/50
 - 5s - loss: 0.0078 - val_loss: 0.0192
Epoch 8/50
 - 6s - loss: 0.0077 - val_loss: 0.0172
Epoch 9/50
 - 6s - loss: 0.0077 - val_loss: 0.0134
Epoch 10/50
 - 6s - loss: 0.0077 - val_loss: 0.0114
Epoch 11/50
 - 5s - loss: 0.0076 - val_loss: 0.0103
Epoch 12/50
 - 6s - loss: 0.0076 - val_loss: 0.0086
Epoch 13/50
 - 6s - loss: 0.0076 - val_loss: 0.0080
Epoch 14/50
 - 6s - loss: 0.0076 - val_loss: 0.0076
Epoch 15/50
 - 6s - loss: 0.0076 - val_loss: 0.0075
Epoch 16/50
 - 6s - loss: 0.0075 - val_loss: 0.0074
Epoch 17/50
 - 5s - loss: 0.0075 - val_loss: 0.0074
Epoch 18/50
 - 6s - loss: 0.0075 - val_loss: 0.0075
Epoch 19/50
 - 6s - los